In [4]:
!nvidia-smi

Sun Mar 17 23:29:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:0A:00.0  On |                  N/A |
|  0%   48C    P5             24W /  215W |     893MiB /   8192MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od
import string
import random
import os

from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize, ToTensor

In [3]:
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)
print(device)

cuda:0


In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, file_path) -> None:
        super().__init__()
        self.data = self.loadData(file_path)
        self.vocab = self.getVocab()
        
    def loadData(self, file_path):
        # Preprocess as well
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        lines = data.split('\n')
        lines = [line.strip(r'\"') for line in lines]
        words = ' '.join(lines).split()
        punct = set(string.punctuation)
        words = [''.join([char for char in list(word) if char not in punct]) for word in words]
        return words
    
    def getVocab(self):
        char_counts = Counter(self.data)
        vocab = {char: idx for idx, (char, _) in enumerate(sorted(char_counts.items()))}
        vocab_size = len(vocab)
        return vocab, vocab_size
    
    def __len__(self):
        return len(self.data)
    
    # def __getitem__(self, index):
    #     return 